# Problem Statement:
A cloth manufacturing company is interested to know about the segment or attributes causes high sale. 
Approach - A Random Forest  can be built with target variable Sale (we will first convert it in categorical variable) & all other variable will be independent in the analysis.

In [7]:
# improrting the necessary libraries 
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [14]:
# importing the dataset
data = pd.read_csv('Company_Data.csv')
data.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [15]:
# data information
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


In [16]:
# Creating copy of a datset
data2 = data.copy()
data2.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [17]:
# descriptive statastics 
data2.describe()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,7.496325,124.975000,68.657500,6.635000,264.840000,115.795000,53.322500,13.900000
std,2.824115,15.334512,27.986037,6.650364,147.376436,23.676664,16.200297,2.620528
min,0.000000,77.000000,21.000000,0.000000,10.000000,24.000000,25.000000,10.000000
25%,5.390000,115.000000,42.750000,0.000000,139.000000,100.000000,39.750000,12.000000
50%,7.490000,125.000000,69.000000,5.000000,272.000000,117.000000,54.500000,14.000000
75%,9.320000,135.000000,91.000000,12.000000,398.500000,131.000000,66.000000,16.000000
max,16.270000,175.000000,120.000000,29.000000,509.000000,191.000000,80.000000,18.000000


In [18]:
# converting catogorical data to numeric data
data2['US'] = data2['US'].map({'Yes':1,'No': 0})
data2['Urban'] = data2['Urban'].map({'Yes':1,'No': 0})
data2['ShelveLoc'] = data2['ShelveLoc'].map({'Bad':0,'Good':2,'Medium':1})
data2

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,0,42,17,1,1
1,11.22,111,48,16,260,83,2,65,10,1,1
2,10.06,113,35,10,269,80,1,59,12,1,1
3,7.40,117,100,4,466,97,1,55,14,1,1
4,4.15,141,64,3,340,128,0,38,13,1,0
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,2,33,14,1,1
396,6.14,139,23,3,37,120,1,55,11,0,1
397,7.41,162,26,12,368,159,1,40,18,1,1
398,5.94,100,79,7,284,95,0,50,12,1,1


In [26]:
# converting the catogorical data to numeric accoring to problem statement
data2['sales'] = 'Small'
data2.loc[data2['Sales'] >= 7.49,'sales'] = 'Large'
data2.drop(['Sales'],axis = 1,inplace = True)

In [27]:
data2.head()

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,sales
0,138,73,11,276,120,0,42,17,1,1,Large
1,111,48,16,260,83,2,65,10,1,1,Large
2,113,35,10,269,80,1,59,12,1,1,Large
3,117,100,4,466,97,1,55,14,1,1,Small
4,141,64,3,340,128,0,38,13,1,0,Small


In [28]:
# input variables
X = data2.iloc[:,0:10]
X

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,138,73,11,276,120,0,42,17,1,1
1,111,48,16,260,83,2,65,10,1,1
2,113,35,10,269,80,1,59,12,1,1
3,117,100,4,466,97,1,55,14,1,1
4,141,64,3,340,128,0,38,13,1,0
...,...,...,...,...,...,...,...,...,...,...
395,138,108,17,203,128,2,33,14,1,1
396,139,23,3,37,120,1,55,11,0,1
397,162,26,12,368,159,1,40,18,1,1
398,100,79,7,284,95,0,50,12,1,1


In [30]:
# Target Variables
Y = data2.iloc[:,10]
Y

0      Large
1      Large
2      Large
3      Small
4      Small
       ...  
395    Large
396    Small
397    Small
398    Small
399    Large
Name: sales, Length: 400, dtype: object

In [46]:
X.shape,Y.shape

((400, 10), (400,))

In [68]:
# Splitting the data on train and test 
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.33,random_state = 0)

In [69]:
# bulding model 
RD = RandomForestClassifier(n_estimators=100,max_features=3)
RD.fit(x_train,y_train)# fitting on train data

RandomForestClassifier(max_features=3)

In [70]:
predict = RD.predict(x_test)# predicting on data
np.mean(predict==y_test)*100 

81.81818181818183

In [59]:
predict

array(['Small', 'Large', 'Large', 'Small', 'Small', 'Large', 'Large',
       'Small', 'Small', 'Small', 'Small', 'Small', 'Small', 'Large',
       'Small', 'Small', 'Large', 'Large', 'Small', 'Small', 'Small',
       'Large', 'Large', 'Large', 'Small', 'Large', 'Small', 'Small',
       'Large', 'Small', 'Large', 'Large', 'Large', 'Small', 'Small',
       'Large', 'Small', 'Small', 'Small', 'Large', 'Small', 'Small',
       'Small', 'Large', 'Small', 'Small', 'Large', 'Large', 'Large',
       'Large', 'Small', 'Large', 'Large', 'Large', 'Small', 'Large',
       'Small', 'Small', 'Small', 'Small', 'Small', 'Large', 'Small',
       'Small', 'Small', 'Large', 'Large', 'Large', 'Large', 'Small',
       'Large', 'Large', 'Large', 'Small', 'Small', 'Large', 'Large',
       'Large', 'Large', 'Large', 'Small', 'Large', 'Small', 'Small',
       'Large', 'Large', 'Small', 'Small', 'Large', 'Large', 'Small',
       'Small', 'Large', 'Small', 'Large', 'Small', 'Large', 'Small',
       'Small', 'Lar

In [60]:
df_t = pd.DataFrame({'Actual':y_test, 'Predicted':predict})
df_t # actual vs predicted

,Actual,Predicted
132,Large,Small
309,Large,Large
341,Small,Large
196,Small,Small
246,Small,Small
...,...,...
252,Large,Small
350,Large,Large
168,Small,Large
150,Large,Large


In [72]:
RD.feature_importances_ # feature importance 

array([0.13186978, 0.0912681 , 0.09294397, 0.09039057, 0.23682868,
       0.13354953, 0.13073442, 0.05624884, 0.01935727, 0.01680883])

In [75]:
# dataframe for better understanding
fi = pd.DataFrame({'feature': list(x_train.columns),
                   'importance': RD.feature_importances_}).\
                    sort_values('importance', ascending = False)
fi

,feature,importance
4,Price,0.236829
5,ShelveLoc,0.133550
0,CompPrice,0.131870
6,Age,0.130734
2,Advertising,0.092944
1,Income,0.091268
3,Population,0.090391
7,Education,0.056249
8,Urban,0.019357
9,US,0.016809


# from above datafram we can see price is the most important feature which is affecting the sales